In [1]:
import pandas as pd
import cptac
import numpy as np
import requests
import json
import deva
import binarization_functions_1 as bf

In [3]:
ov = cptac.Ovarian()
clinical = ov.get_clinical()
proteomics = ov.get_proteomics()
follow_up = pd.read_excel('../Follow_Up_Data/Ovary_One_Year_Clinical_Data_20160927.xls')

In [7]:
for col in follow_up.columns:
    if len(follow_up[col].value_counts()) <= 1:
        follow_up = follow_up.drop(col, axis=1)
    else:
        print(col)
        print(follow_up[col].value_counts())
        print('\n')

Short Title
CPTAC-MS-04OV    120
CPTAC-WU-01OV     64
CPTAC-CS-02OV     54
CPTAC-UO-17OV     33
CPTAC-UP-26OV     16
CPTAC-UK-13OV      8
CPTAC-JC-20OV      7
CPTAC-AE-15OV      3
CPTAC-HF-22OV      3
CPTAC-CL-18OV      2
CPTAC-MO-14OV      2
CPTAC-AS-11OV      1
Name: Short Title, dtype: int64


PPID
01OV018    8
01OV045    8
20OV005    7
26OV008    6
13OV003    5
          ..
17OV017    1
17OV029    1
17OV033    1
17OV012    1
17OV038    1
Name: PPID, Length: 114, dtype: int64


Event
Collection     310
Pre Surgery      3
Name: Event, dtype: int64


Modified Time
2016-04-13    56
2016-04-22    50
2016-05-03    35
2016-03-21    32
2016-04-11    30
2016-03-25    28
2016-02-17    15
2016-04-25    15
2016-04-21    14
2016-06-23     5
2016-03-24     4
2016-06-15     4
2016-03-22     4
2016-04-28     3
2016-03-17     3
2016-05-02     3
2016-03-18     2
2015-07-07     1
2015-07-13     1
2015-11-06     1
2016-04-05     1
2015-06-17     1
2015-10-29     1
2016-01-25     1
2016-03-07     1
201

In [9]:
cols_to_drop = ['Short Title', 'PPID', 'Event', 'Modified Time', 
                'Modified By', 'Status', 'CRF Name', 
                'Was a Review of the Initial Pathological Findings Done?', 
                'Was the Pathology Review consistent with the Diagnosis?', 
                'Adjuvant (Post-Operative) Radiation Therapy', 
                'Adjuvant (Post-Operative) Immunotherapy', 
                'Adjuvant (Post-Operative) Hormone Therapy', 
                'Anatomic Site of New Tumor Event', 
                'Other Site of New Tumor Event or Lymph Node Location', 
                'Method Of Diagnosis of New Tumor Event', 
                'Date of Additional Surgery for New Tumor Event', 
                'Additional Hormone Therapy Treatment of New Tumor Event', 
                'Additional Targeted Molecular Therapy Treatment of New Tumor Event', 
                'What Type of Malignancy was This?', 'Primary Site of Disease', 
                'Laterality of the Disease', 'Did the patient have surgery for this malignancy?',
                'Number of Days from Date of Initial Pathologic Diagnosis of the Tumor Submitted for CPTAC to the Date of Initial Diagnosis of Other Malignancy', 
                'Did the patient receive pharmaceutical therapy for this malignancy?', 
                'Was the patient staged using FIGO?', ' Was the patient staged using AJCC?']

follow_up_annotations = follow_up.drop(cols_to_drop, axis=1).copy()

things_to_replace = ['Not Reported/ Unknown', 'Reported/ Unknown', 
                     'Not Applicable', 'na', 'unknown', 'Not Performed', 
                     'Unknown tumor status', 'Unknown']

for col in follow_up_annotations.columns:
    follow_up_annotations[col] = follow_up_annotations[col].replace(things_to_replace, np.nan)

for col in follow_up_annotations.columns:
    if np.issubdtype(follow_up_annotations[col].dtype, np.number):
        mean = follow_up_annotations[col].mean()
        follow_up_annotations[col]= bf.binarizeCutOff(follow_up_annotations, col, mean,
                                                      "Above_Mean("+str(round(mean, 2))+")",
                                                      "Below_Mean("+str(round(mean, 2))+")")

In [11]:
for col in follow_up_annotations.columns:
    print(col)
    print(follow_up_annotations[col].value_counts())
    print('\n')

Date of Last Contact (Do not answer if patient is deceased)
2015-05-26    4
2015-11-20    3
2016-04-20    3
2015-05-22    3
2016-03-10    3
             ..
2016-03-24    1
2016-01-22    1
2015-12-22    1
2015-06-19    1
2014-01-24    1
Name: Date of Last Contact (Do not answer if patient is deceased), Length: 81, dtype: int64


Vital Status (at time of last contact)
Living      104
Deceased     10
Name: Vital Status (at time of last contact), dtype: int64


Date of Death
2015-05-04    1
2014-10-16    1
2014-11-08    1
2012-02-09    1
2015-04-01    1
2015-06-05    1
2013-12-28    1
2013-11-16    1
2014-05-30    1
2015-09-02    1
Name: Date of Death, dtype: int64


Tumor Status at Time of Last Contact or Death
Tumor free    52
With tumor    51
Name: Tumor Status at Time of Last Contact or Death, dtype: int64


Adjuvant (Post-Operative) Pharmaceutical Therapy
Yes    103
No       8
Name: Adjuvant (Post-Operative) Pharmaceutical Therapy, dtype: int64


Adjuvant (Post-Operative) Targeted Mol